In [1]:
# Data processing
import numpy as np
import pandas as pd

# Download data
import yfinance as yf

In [2]:
# Initial investment and currency per stock
investment = {
    "^VIX" : [1000000, 'USD'],
    "^DJI" : [2500000, 'USD'],
    "^AEX"  : [500000, 'EUR'],
    "^GSPC" : [2300000, 'USD'],
    "^N225" : [1750000, 'JPY']
}

# Define whole-year period only
start = "2010-01-01"
end = "2018-12-31"

In [3]:
# Download data and store in dataframe
stockprice_data = yf.download(' '.join(list(investment.keys())), start=start, end=end, groupby='ticker')
stockprice = stockprice_data['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [4]:
stockprice.tail()

,^AEX,^DJI,^GSPC,^N225,^VIX
Date,,,,,
2018-12-24,478.339996,21792.199219,2351.100098,NaN,36.070000
2018-12-25,NaN,NaN,NaN,19155.740234,NaN
2018-12-26,NaN,22878.449219,2467.699951,19327.060547,30.410000
2018-12-27,476.029999,23138.820312,2488.830078,20077.619141,29.959999
2018-12-28,484.170013,23062.400391,2485.739990,20014.769531,28.340000


In [5]:
# Create exchange rate ticker list based on initial investment data
currencies = list(set(['EUR' + i[1] + '=X' for i in investment.values()]))

# Remove EUR-EUR exchange rate
for currency in currencies:
    if currency[0:2] == currency[3:5]:
        currencies.remove(currency)
        
currencies = " ".join(currencies)

In [6]:
# Download exchange rate data
exchangerate_data = yf.download(currencies, start=start, end=end, groupby='ticker')
exchangerate = exchangerate_data['Adj Close']
exchangerate['EUREUR=X'] = 1
exchangerate.tail()

[*********************100%***********************]  2 of 2 completed


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,EURJPY=X,EURUSD=X,EUREUR=X
Date,,,
2018-12-24,126.292000,1.137281,1
2018-12-25,125.769997,1.140394,1
2018-12-26,126.001999,1.141553,1
2018-12-27,126.404999,1.136131,1
2018-12-28,126.767998,1.143105,1


In [7]:
# Scrape EURIBOR rates
import csv
import requests
import io

year_range = int(end[0:4]) - int(start[0:4]) + 1
years = range(int(start[0:4]), int(start[0:4]) + year_range)

# Get EURIBOR rates from the website of the European Money Market Institute
urls = ['https://www.emmi-benchmarks.eu/assets/components/rateisblue/file_processing/publication/processed/hist_EURIBOR_{}.csv'
        .format(i) for i in range(int(start[0:4]), int(start[0:4]) + year_range)]

# Retrieve data from CSVs from the website and store in a pandas dataframe
for url in urls:
    s = requests.get(url).content
    if url == urls[0]:
        euribor = pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose()
    else:
        euribor = euribor.append(pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose())

# Skip lines that have an error code
error_lines = ['Unnamed: 0', 'Unnamed: 261']

euribor3m = euribor[~euribor.index.isin(error_lines)]

In [8]:
# Rename, resize and change index to datetime
euribor3m.index = euribor3m.index.rename('Date')
euribor3m = euribor3m.rename(columns={5: 'EURIBOR3M'})
euribor3m = euribor3m['EURIBOR3M']
euribor3m.index = pd.to_datetime(euribor3m.index, format='%d/%m/%Y')

In [9]:
# Merge three dataframes: the stock prices, exchange rates and three-month EURIBOR
from functools import reduce

dfs = [stockprice, exchangerate, euribor3m]
df = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
df.tail()

,^AEX,^DJI,^GSPC,^N225,^VIX,EURJPY=X,EURUSD=X,EUREUR=X,EURIBOR3M
Date,,,,,,,,,
2018-12-20,485.970001,22859.599609,2467.419922,20392.580078,28.379999,128.018005,1.138952,1,-0.238
2018-12-21,484.809998,22445.369141,2416.620117,20166.189453,30.110001,127.351997,1.145685,1,-0.238
2018-12-24,478.339996,21792.199219,2351.100098,NaN,36.070000,126.292000,1.137281,1,-0.237
2018-12-27,476.029999,23138.820312,2488.830078,20077.619141,29.959999,126.404999,1.136131,1,-0.237
2018-12-28,484.170013,23062.400391,2485.739990,20014.769531,28.340000,126.767998,1.143105,1,-0.236


In [10]:
# Fill missing values with average of adjacent cells
df = (df.ffill() + df.bfill()) / 2

In [17]:
# Convert the financial products denoted in foreign currency to euro
portfolio = df

for column in portfolio.columns:
    if column.startswith('^'):
        portfolio[str(column[1:len(column)])] = portfolio[column] / portfolio['EUR{}=X'.format(investment.get(column)[1])]

In [18]:
portfolio

,^AEX,^DJI,^GSPC,^N225,^VIX,EURJPY=X,EURUSD=X,EUREUR=X,EURIBOR3M,AEX,DJI,GSPC,N225,VIX
Date,,,,,,,,,,,,,,
2010-01-04,343.029999,10583.959961,1132.989990,10654.790039,20.040001,133.389999,1.442398,1.0,0.700,343.029999,7337.753649,785.490635,79.876978,13.893532
2010-01-05,342.019989,10572.019531,1136.520020,10681.830078,19.350000,131.529999,1.436596,1.0,0.699,342.019989,7359.076695,791.120180,81.212120,13.469341
2010-01-06,341.480011,10573.679688,1137.140015,10731.450195,19.160000,133.013000,1.440403,1.0,0.694,341.480011,7340.776974,789.459439,80.679709,13.301830
2010-01-07,340.549988,10606.860352,1141.689941,10681.660156,19.059999,133.748001,1.431803,1.0,0.692,340.549988,7408.043337,797.379081,79.864073,13.311885
2010-01-08,341.940002,10618.190430,1144.979980,10798.320312,18.129999,133.490005,1.441109,1.0,0.691,341.940002,7368.068604,794.513067,80.892350,12.580588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-20,485.970001,22859.599609,2467.419922,20392.580078,28.379999,128.018005,1.138952,1.0,-0.238,485.970001,20070.728950,2166.394745,159.294624,24.917640
2018-12-21,484.809998,22445.369141,2416.620117,20166.189453,30.110001,127.351997,1.145685,1.0,-0.238,484.809998,19591.216579,2109.322765,158.350005,26.281214
2018-12-24,478.339996,21792.199219,2351.100098,20121.904297,36.070000,126.292000,1.137281,1.0,-0.237,478.339996,19161.663459,2067.298870,159.328416,31.715991
